In [ ]:
import numpy as np
import evaluate

metric = evaluate.load("seqeval")

Using the latest cached version of the module from C:\Users\Anacl\.cache\huggingface\modules\evaluate_modules\metrics\evaluate-metric--seqeval\541ae017dc683f85116597d48f621abc7b21b88dc42ec937c71af5415f0af63c (last modified on Tue Apr 18 00:53:25 2023) since it couldn't be found locally at evaluate-metric--seqeval, or remotely on the Hugging Face Hub.


In [16]:
def compute_metrics(p):
        predictions, labels = p
        predictions = np.argmax(predictions, axis=2)

        true_predictions = [
            [label_list[p] for (p, l) in zip(prediction, label) if l != -100]
            for prediction, label in zip(predictions, labels)
        ]
        true_labels = [
            [label_list[l] for (p, l) in zip(prediction, label) if l != -100]
            for prediction, label in zip(predictions, labels)
        ]
        results = metric.compute(predictions=true_predictions, references=true_labels)
        final_results = {}
        for key, value in results.items():
            if isinstance(value, dict):
                for n, v in value.items():
                    final_results[f"{key}_{n}"] = v
            else:
                final_results[key] = value
        return final_results


In [17]:
from datasets import load_from_disk
from transformers import AutoModelForTokenClassification, Trainer, AutoTokenizer, DataCollatorForTokenClassification, TrainingArguments

dataset = load_from_disk("./data/ncbi.hf")
dataset
features = dataset["train"].features
label_list = features['ner_tags'].feature.names

In [18]:
model = AutoModelForTokenClassification.from_pretrained(
            "./models/BERT", 
            local_files_only=True,
            use_auth_token=False,
        )

tokenizer = AutoTokenizer.from_pretrained("./models/BERT", local_files_only=True, padding=True)

data_collator = DataCollatorForTokenClassification(tokenizer)

trainer = Trainer(
            model=model,
            train_dataset= dataset['train'],
            tokenizer=tokenizer,
            data_collator=data_collator,
            compute_metrics=compute_metrics,
            args = TrainingArguments(optim="adamw_torch", num_train_epochs=5, output_dir = str("./output"), evaluation_strategy="no", save_strategy="no", do_eval=False)
    )

c:\Users\Anacl\Documents\GitHub\COSINER\lib\site-packages\transformers\models\auto\auto_factory.py:472: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers. Please use `token` instead.
  warnings.warn(
Some weights of BertForTokenClassification were not initialized from the model checkpoint at ./models/BERT and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
